In [ ]:
!pip install flask_wtf
!pip install flask_login
!pip install flask_bcrypt
!pip install email_validator
!pip install flask-admin

In [ ]:
################################################################################################################

# EXTERNAL MODULES TO BE USED

################################################################################################################
from flask import Flask, render_template, redirect, url_for, session, flash, request
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
import pandas as pd
from flask_admin import Admin,BaseView,expose,AdminIndexView
from flask_admin.contrib.sqla import ModelView

app = Flask(__name__)

db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config["SECRET_KEY"] = "1234567"
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database1.db'


login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

def is_accessible(self):
    return current_user.is_authenticated

class User(db.Model, UserMixin):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True,autoincrement=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    to_do_tasks = db.Column(db.String(80))
    expected_due = db.Column(db.String(80))
    
    def __repr__(self):
        return '<User %r>' % self.id    
    
admin = Admin(app)
class MyV1(ModelView):
    def is_accessible(self):
        if current_user.is_authenticated:
            return True
    
    @property
    def can_create(self):
         return login.current_user.login == '1111'
        
    column_list = ('id', 'to_do_tasks','expected_due')
    def __init__(self, session, **kwargs):
        super(MyV1, self).__init__(User, session, **kwargs)

admin.add_view(MyV1(db.session,name = 'final-project'))


class RegisterForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(), Email(message="Invalid Email"), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")
            
class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")
    


@app.route("/users")
@login_required
def users():
    try:
        users = User.query.order_by(User.id.asc()).all()
        print("Entry successfully retrieved.")
    except Exception as e:
        print("Exception occurred.",e)
    list = []
    for user in users:
        print(user.id, user.username, user.email)
        dict = {'id':user.id, 'username':user.username, 'email':user.email}
        print(dict)
        # dict_copy = dict.copy()
        list.append(dict)

    return render_template('users.html', entries = list)

################################################################################################################

# WEB ROUTES FOR CONTROLLING ACCESS TO TEMPLATE VIEWS

################################################################################################################

@app.route("/")
def index():
    return render_template('index.html')


@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for('view_index'))    
            
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)


@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data) ##
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)
    
    
@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))

@app.route('/', methods=['GET','POST'])
@login_required
def view_index():
    return render_template('index.html', title='My profile')

@app.route("/store")
@login_required
def store():
    df = pd.read_csv("diy-shop2.csv")
    list = df.to_dict('records')

    return render_template('store.html', entries = list)


################################################################################################################

# ERROR HANDLERS

################################################################################################################

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404


################################################################################################################

# APPLICATION TEST RUN AT PORT 9003

################################################################################################################

if __name__ == '__main__':
    app.run('localhost', 9040)
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/peiwei/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:851: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  warnings.warn(
/Users/peiwei/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9040/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Oct/2021 11:43:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 11:43:14] "GET /admin HTTP/1.1" 308 -
127.0.0.1 - - [27/Oct/2021 11:43:14] "GET /admin/ HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 11:43:14] "GET /admin/static/bootstrap/bootstrap2/swatch/default/bootstrap.min.css?v=2.3.2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 11:43:14] "GET /admin/static/admin/css/b